In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from src.analysis.fraudulous import detect_honey_frauders
from src.constants import HONEYPOTS, LOG_FORMS_RESULTS_DIR
import pandas as pd
import Levenshtein
from pathlib import Path
import pickle as pk
from src.data.postprocessing import (get_emojis_voc_counts,write_emojis_voc_counts,
                                     display_whole_dir,generate_production_format)
import numpy as np
import pickle as pk

# Postprocessing

<div class="alert alert-success">
Postprocessing of data gathered from mturk/gform
</div>

In [5]:
# easy visualization of the gathered dataframes
display_whole_dir(Path("../data/processed/auto_mturk/forms_log_results/run4"))

,Timestamp,WorkerID,🔈,🥜,😆,📭,🎞️,☂️,💣,🛐,🌸,🛣️,📡,Feedback
0,02/12/2020 14:50:48,A30UE6DWFNUCWX,volume,peanut,laughing,mailbox,film,umbrella,bomb,praying,flower,highway,satellite,it was fun thanks so much
1,02/12/2020 14:51:58,AWN1PEZWRQSQ9,volume,peanut,laughter,mailbox,film,umbrella,bomb,church,flower,exit,satalight,NaN
2,02/12/2020 14:52:10,A18T7E73TNGOKP,speaker,peanut,laughing,mailbox,film,umbrella,bomb,pray,flower,freeway,satellite,no
3,02/12/2020 14:53:01,A2YO837C0O1E91,volume,peanut,laughing,mailbox,film,umbrella,bomb,pray,flower,highway,cable,NaN
4,02/12/2020 14:59:56,A2D2JX8R0QU9G4,speaker,peanut,laugh,mailbox,flim,umbrella,bomb,worship,flower,highway,antenna,Nothing
5,02/12/2020 15:01:44,A12OZ7P7Z41SJI,sound,peanuts,laugh,mail,video,rain,bomb,shelter,flower,highway,satelite,NaN
6,02/12/2020 15:02:19,A143XRCI1YXAFE,volume,peanut,laugh,mail,film,umbrella,bomb,pray,lilly,highway,satelite,NaN
7,02/12/2020 15:07:34,A2ALESYWNEZU1D,speaker,peanut,laugh,mailbox,flim,umbrella,bomb,worship,flower,highway,antenna,Nothing
8,02/12/2020 15:09:09,A13OMD8QDV0YON,sound,peanut,lfunny,mail,movie,umbrella,bomb,pray,flower,highway,signal,NaN
9,02/12/2020 16:16:47,AAQREZOK13OV7,sound,peanut,laughing,mail,film,umbrella,bomb,prayer,flower,driving,satellite,NaN


,Timestamp,WorkerID,👨‍💼,🛋️,🥴,🖲️,🤤,⭐,🥘,💊,🔨,😮,📟,Feedback
0,02/12/2020 14:50:37,AWN1PEZWRQSQ9,office,couch,crazy,space,drool,star,stewpot,pills,hammer,wow,pager,NaN
1,02/12/2020 14:52:51,A30UE6DWFNUCWX,employee,couch,nervous,knight,drooling,star,soup,pill,hammer,shocked,pager,thank you!
2,02/12/2020 14:53:25,A18T7E73TNGOKP,presentation,livingroom,drunk,ufo,drooling,star,food,pill,hammer,shocked,creditcard,NaN
3,02/12/2020 15:01:03,A345VDWYMDWTGD,stocks,furniture,drunk,button,drooling,star,skillet,pill,hammer,wow,card,NaN
4,02/12/2020 15:01:59,A2D2JX8R0QU9G4,manager,sofa,guilt,bot,vomit,star,cook,capsule,hammer,exclamation,calculator,Nothing
5,02/12/2020 15:10:40,A13OMD8QDV0YON,work,couch,confused,alien,funny,star,food,medicine,tool,surprised,pager,NaN
6,02/12/2020 15:11:32,A1PLZ8RM4NW43J,businessman,couch,awkward,button,drool,star,stew,pill,hammer,wow,pager,NaN
7,02/12/2020 15:12:11,A143XRCI1YXAFE,presentation,sofa,silly,button,yum,star,paella,pill,hammer,wow,pager,NaN
8,02/12/2020 15:23:38,A15OKHGB4RDA9P,happy,couch,sick,button,happy,star,lunch,tablet,hammer,wow,calculator,good
9,02/12/2020 15:44:40,A12OZ7P7Z41SJI,teacher,home,crazy,button,drool,star,dinner,medicine,hammer,ohh,debit,NaN


## Visualizing the vocabulary 

In [4]:
em2vocab = get_emojis_voc_counts(Path("../data/processed/auto_mturk/forms_log_results/"))

In [5]:
write_emojis_voc_counts(em2vocab,"./emvoc_500ass.txt")

## Generating final format

In [11]:
for path in Path("../data/processed/auto_mturk/forms_results/18/").glob("[0-9]*.csv"):
    df = pd.read_csv(path)
    number = path.stem
    print(f"df {number}: {df.shape[0]} rows")

df 127: 30 rows
df 126: 30 rows
df 132: 30 rows
df 130: 31 rows
df 131: 30 rows
df 128: 30 rows
df 129: 30 rows


In [19]:
prod_path = Path("../data/processed/auto_mturk/forms_results/")
total_df = generate_production_format(prod_path)
total_df.to_csv("../data/processed/auto_mturk/prod_format/prod/emoji_dataset.csv")

100%|██████████| 133/133 [00:03<00:00, 37.81it/s]


In [23]:
total_df

,WorkerID,FormId,Duration,emoji_index,emoji,word
5403,AU2RMH9IZP60M,65,802.0,0,#️⃣,number
5373,A1XHK92L7Q7VEU,65,549.0,0,#️⃣,hashtag
5253,A31UXXZVI3U4E2,65,155.0,0,#️⃣,pound
5263,A3RYI5HXC2MJLN,65,592.0,0,#️⃣,number
5443,A70TBKPGWQZO4,65,203.0,0,#️⃣,pound
...,...,...,...,...,...,...
25837,A13OMD8QDV0YON,71,139.0,1324,🪕,banjo
25827,A2C7A6E70NYNUI,71,156.0,1324,🪕,banjo
25817,A2MOKIEQZ0OF2M,71,827.0,1324,🪕,banjo
25967,A14OPFM8OFA4WF,71,674.0,1324,🪕,banjo


In [12]:

paths = sorted(prod_path.glob("[0-9]*"),key= lambda x: int(x.stem))
for path in paths:
    if len(list(path.iterdir())) > 0:
        batch_number = path.stem
        print(f"Batch {batch_number} computation")
        df = generate_production_format(path)
        df.to_csv(Path("../data/processed/auto_mturk/prod_format/prod").joinpath(batch_number+ ".csv"))

  0%|          | 0/7 [00:00<?, ?it/s]

Batch 0 computation


 57%|█████▋    | 4/7 [00:00<00:00, 34.86it/s]

Batch 1 computation


  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

Batch 17 computation


 57%|█████▋    | 4/7 [00:00<00:00, 38.26it/s]

Batch 18 computation


100%|██████████| 7/7 [00:00<00:00, 35.49it/s]


In [17]:
df.sample(10)

,WorkerID,FormId,Duration,emoji_index,emoji,word
1608,A1J7VD5KF0A442,128,406.0,454,🐊,animal
1334,A2A6FH0F7LD9ND,131,290.0,573,👨‍👩‍👦,family
1510,AAQREZOK13OV7,128,927.0,716,📁,folder
367,A3BZVFD82AVLT8,126,414.0,1241,🦻,headphone
1629,A2RWLFIP23FZA7,128,655.0,1209,🦖,dinosaur
1666,A143XRCI1YXAFE,128,954.0,950,😼,cat
1452,A2C7A6E70NYNUI,128,334.0,141,❇️,star
28,A82WUWI7H3MZWA,127,NaN,708,💹,symbol
153,A1PBRKFHSF1OF8,127,210.0,1050,🛢️,barrel
476,A2T5CROJ0FWIJY,126,431.0,774,📻,radio


In [18]:
prod_path= Path("../data/processed/auto_mturk/forms_results/18")
comments = set()
for path in prod_path.glob("**/[0-9]*.csv"):
    df = pd.read_csv(path)
    comments.update(df['Feedback'])

comments

{' ',
 'None.',
 'Please just make the timer 1 hour instead of incrementally increasing it by 2 minutes every time!!!!',
 'THANKS, THE ROUND WHEEL-LIKE OBJECT WAS DIFFICULT',
 'good',
 'i like it',
 'i like that survey',
 'i tried my best',
 nan,
 'nice',
 'none',
 'some were difficult',
 'thank you',
 'thanks'}

In [121]:
df = pd.read_csv("../data/processed/auto_mturk/forms_results/13/96.csv")

In [122]:
df[df.Age == 28]

,Timestamp,WorkerID,Age,Gender,Mothertongue,🖌️,♓,☹️,📖,☪️,🍋,🍱,👨‍❤️‍👨,💦,🤔,🙍,Feedback
0,17/12/2020 23:58:24,AXXA481XH6FKE,28.0,Female,english,brush,zodiac,sad,book,moon,lemon,sushi,couple,drop,thinking,sad,NaN
25,18/12/2020 00:29:13,A2RWLFIP23FZA7,28.0,Male,chinese,pen,pisces,sad,book,moonstar,lemon,lunchbox,love,drops,thinking,adult,NaN
29,18/12/2020 01:31:02,A70TBKPGWQRO4,28.0,Female,english,brush,zodiac,upset,book,croissant,lemon,japanese,love,drops,perplex,unsettle,thanks


In [50]:
winfo = pd.read_csv("../data/processed/auto_mturk/forms_results/1/workers_info.csv")

In [54]:
import Levenshtein

In [59]:
mask = winfo.WorkerId.apply(lambda x:Levenshtein.distance("A3FBC5W56LKQXL",x) < 1)

In [64]:
"A3FBC5W56LKQXL" in winfo['WorkerId'].values

True

In [60]:
winfo[mask]

,WorkerId,FormId,AnswerDurationInSeconds
50,A3FBC5W56LKQXL,12,337
109,A3FBC5W56LKQXL,10,465
139,A3FBC5W56LKQXL,11,592
166,A3FBC5W56LKQXL,7,223


In [78]:
temp = pd.read_csv("../data/processed/auto_mturk/forms_results/1/11.csv")

temp.shape

(31, 17)

In [75]:
df[(df.Duration.isna())][["WorkerID","FormId"]].drop_duplicates()

,WorkerID,FormId
868,A3FBC5W56LKQXL,13
2103,AUTBAO7DP20SV,12
1993,A82WUWI7H3MZWA,12
1354,A2KBTDHM44J7X4,10
426,A2D2JX8R0QU9G4,11


In [49]:
df.sample(10)

,WorkerID,FormId,Duration,emoji_index,emoji,word
2067,A1TW2BZRRS874Z,12,300.0,1164,🥥,coconut
861,A3FBC5W56LKQXL,13,NaN,259,🌼,daisy
1398,A2XE89KFI1DRPQ,10,596.0,373,🎴,sunset
1893,A3VG22WR5I4UG2,12,195.0,104,⛈️,thunder
920,A1PBRKFHSF1OF8,7,44.0,150,➕,plus
1252,A13OMD8QDV0YON,10,356.0,256,🌹,flower
939,A1UE6IH3L3XYN2,7,54.0,1070,🟤,circle
481,A4VUQ6627RJX8,11,122.0,1291,🧮,abacus
1796,A2BYI7MFOB0ZTI,9,390.0,399,🏏,bat
524,A18QKM5Z69TLS,11,241.0,979,🚉,train


In [13]:
df1 = generate_production_format(Path("../data/processed/auto_mturk/forms_log_results/demographicrun5//"))
pk.dump(df1,open("./demographic_pilot5.pk","wb"))

100%|██████████| 1/1 [00:00<00:00, 38.80it/s]
